In [1]:
import os
os.environ["JAVA_HOME"] = "/mnt/extproj/projekte/textmining/jdk/openlogic-openjdk-11.0.22+7-linux-x64"

from collections import Counter

In [2]:
from collections import defaultdict
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import Window
import glob

In [3]:
if not "spark" in globals() or spark is None:
    print("Creating builder")
    spark = SparkSession.builder.\
            config("spark.executor.memory", "70g").\
            config("spark.driver.memory", "50g").\
            config("spark.memory.offHeap.enabled",True).\
            config("spark.memory.offHeap.size","16g").\
            config("spark.sql.shuffle.partitions", 300).\
            appName('mirexplore').getOrCreate()
    
#spark.conf.set("spark.executor.memory", "70g")
#spark.conf.set("spark.driver.memory", "50g")
#spark.conf.set("spark.memory.offHeap.enabled",True)
#spark.conf.set("spark.memory.offHeap.size","16g")  
#spark.conf.set("spark.sql.shuffle.partitions", 300)

Creating builder


/mnt/extproj/projekte/bartelt/software/miniconda3/envs/mirexplore/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
df_int = spark.read.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_mirna_gene_consensus_parquet/")
df_int = df_int.withColumn("doc_id", explode("evidence_documents"))

In [5]:
relDocIDs = set([x["doc_id"] for x in df_int.select("doc_id").collect()])
print(len([x for x in relDocIDs if x.startswith("PMC")]))

55331


In [6]:
def read_date_file(inputpath, relevantDocIDs):
    db_entries = []
    ignoredConcepts = Counter()


    for infile in glob.glob("{}/*.date".format(inputpath)):
        with open(infile, "r") as fin:
            
            for iline, line in enumerate(fin):
                aline = line.split("\t")

                docID = aline[0]
                
                if not docID in relevantDocIDs:
                    continue
                
                if len(aline) < 4:
                    continue
                
                year, month, day = int(aline[1]), int(aline[2]), int(aline[3])
                if day == 0:
                    day = 1
                if month == 0:
                    month = 1
                
                db_entries.append( (docID, "{:04d}-{:02d}-{:02d}".format(year, month, day)) )
                
    return db_entries


In [7]:
docSentences = list(set(read_date_file("/mnt/extproj/projekte/textmining/pubmed_feb24/", relDocIDs)))
print(len(docSentences))

47280


In [8]:
docSentencesPMC = list(set(read_date_file("/mnt/extproj/projekte/textmining/pmc_feb24/oa_comm/", relDocIDs)))
print(len(docSentencesPMC))

55334


In [9]:
df = spark.createDataFrame(docSentences+docSentencesPMC, ["doc_id", "date"])

In [10]:
df.show()

+--------+----------+
|  doc_id|      date|
+--------+----------+
|35107490|2022-01-31|
|32583425|2020-06-24|
|28056303|2016-12-08|
|33850858|2021-03-01|
|24111776|2013-11-22|
|37963060|2023-11-14|
|25887381|2015-04-17|
|25339353|2014-10-22|
|36339585|2022-10-19|
|26617770|2015-09-01|
|33777215|2021-03-18|
|37326716|2023-01-01|
|26540659|2015-11-01|
|37300283|2023-12-01|
|35171545|2022-01-01|
|32233464|2020-04-01|
|35809316|2022-05-22|
|36276867|2022-10-14|
|28138701|2017-01-24|
|31840934|2019-12-16|
+--------+----------+
only showing top 20 rows



In [11]:
df.write.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_document_pubdates", mode="overwrite")